In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
file_path = '/content/drive/MyDrive/semester 2/predic_tabel.csv'
df = pd.read_csv(file_path)

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Label encoder per kolom
encoders = {
    'Jenis_Kelamin': LabelEncoder(),
    'Merokok': LabelEncoder(),
    'Bekerja': LabelEncoder(),
    'Rumah_Tangga': LabelEncoder(),
    'Aktivitas_Begadang': LabelEncoder(),
    'Aktivitas_Olahraga': LabelEncoder(),
    'Asuransi': LabelEncoder(),
    'Penyakit_Bawaan': LabelEncoder(),
    'Hasil': LabelEncoder()
}

for col, encoder in encoders.items():
    df[col] = encoder.fit_transform(df[col])

df['Usia'] = df['Usia'].map({'Muda': 0, 'Tua': 1})

X = df.drop(columns=['No', 'Hasil'])
y = df['Hasil']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [6]:
print("=== FORM CEK RISIKO PENYAKIT PARU-PARU ===")
usia = input("1. Usia Anda? (Muda/Tua): ")
jenis_kelamin = input("2. Jenis kelamin? (Pria/Wanita): ")
merokok = input("3. Status merokok? (Aktif/Pasif/Tidak): ")
bekerja = input("4. Apakah sedang bekerja? (Ya/Tidak): ")
rumah_tangga = input("5. Apakah sudah berumah tangga? (Ya/Tidak): ")
begadang = input("6. Apakah sering begadang? (Ya/Tidak): ")
olahraga = input("7. Seberapa sering olahraga? (Jarang/Sering): ")
asuransi = input("8. Apakah punya asuransi? (Ada/Tidak): ")
penyakit_bawaan = input("9. Apakah punya penyakit bawaan? (Ada/Tidak): ")

input_data = {
    'Usia': 1 if usia.lower() == 'tua' else 0,
    'Jenis_Kelamin': encoders['Jenis_Kelamin'].transform([jenis_kelamin.title()])[0],
    'Merokok': encoders['Merokok'].transform([merokok.title()])[0],
    'Bekerja': encoders['Bekerja'].transform([bekerja.title()])[0],
    'Rumah_Tangga': encoders['Rumah_Tangga'].transform([rumah_tangga.title()])[0],
    'Aktivitas_Begadang': encoders['Aktivitas_Begadang'].transform([begadang.title()])[0],
    'Aktivitas_Olahraga': encoders['Aktivitas_Olahraga'].transform([olahraga.title()])[0],
    'Asuransi': encoders['Asuransi'].transform([asuransi.title()])[0],
    'Penyakit_Bawaan': encoders['Penyakit_Bawaan'].transform([penyakit_bawaan.title()])[0],
}


user_df = pd.DataFrame([input_data])
hasil_prediksi = encoders['Hasil'].inverse_transform(model.predict(user_df))[0]

print(f"\n=== HASIL ANALISIS ===")
print(f"Status Risiko Penyakit Paru-paru Anda: 🚨 {hasil_prediksi.upper()} 🚨")

=== FORM CEK RISIKO PENYAKIT PARU-PARU ===
1. Usia Anda? (Muda/Tua): muda
2. Jenis kelamin? (Pria/Wanita): pria
3. Status merokok? (Aktif/Pasif/Tidak): aktif
4. Apakah sedang bekerja? (Ya/Tidak): tidak
5. Apakah sudah berumah tangga? (Ya/Tidak): tidak
6. Apakah sering begadang? (Ya/Tidak): ya
7. Seberapa sering olahraga? (Jarang/Sering): jarang
8. Apakah punya asuransi? (Ada/Tidak): tidak
9. Apakah punya penyakit bawaan? (Ada/Tidak): tidak

=== HASIL ANALISIS ===
Status Risiko Penyakit Paru-paru Anda: 🚨 TIDAK 🚨


In [8]:
%pip install -U langchain-community replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [12]:
from langchain_community.llms import Replicate
from google.colab import userdata
import os

os.environ["REPLICATE_API_TOKEN"] = userdata.get("api_token")

llm = Replicate(
  model="ibm-granite/granite-3.3-8b-instruct",
  input={"top_k": 5, "top_p": 1.0, "max_tokens": 4096, "temperature": 0.7}
)

prompt = f"""
Berikut adalah data pasien:

- Usia: {usia}
- Jenis Kelamin: {jenis_kelamin}
- Merokok: {merokok}
- Bekerja: {bekerja}
- Rumah Tangga: {rumah_tangga}
- Sering Begadang: {begadang}
- Olahraga: {olahraga}
- Asuransi: {asuransi}
- Penyakit Bawaan: {penyakit_bawaan}

Model kami memprediksi: {hasil_prediksi}

Analisis data di atas dan jawab 3 poin secara ringkas:
1. Risiko penyakit paru-paru (Ya/Tidak) + alasan.
2. 2–3 faktor risiko utama.
3. 1–2 saran tindakan atau pencegahan.
"""

output = llm(prompt)
print("\n=== Analisis AI Granite ===")
print(output)



=== Analisis AI Granite ===
1. Risiko penyakit paru-paru (Ya): Berdasarkan data yang diberikan, risiko penyakit paru-paru untuk individu tersebut adalah Ya. Alasan utamanya adalah karena merokok aktif (merokok adalah faktor utama risiko penyakit paru-paru), sering begadang (mungkin menunjukkan kerap digempar oleh kecelakaan atau keadaan berisiko, yang dapat menyebabkan trauma paru-paru), dan jarang melakukan olahraga (kekurangan aktivitas fisik dapat meningkatkan risiko penyakit paru-paru).

2. Faktor risiko utama:
   - Merokok aktif: Penghabisan rokok aktif sangat terkait dengan penyakit paru-paru obstruktif chronis (PCO), bronitis, dan asma.
   - Sering begadang: Trauma paru-paru mungkin menyebabkan komplikasi paru-paru, termasuk infeksi paru-paru berulang dan paru-paru kolaps.
   - Jarang melakukan olahraga: Kekurangan aktivitas fisik dapat membantu penyakit-penyakit paru-paru kronis karena paru tidak dapat dibejakan secara efisien, menghasilkan kekurangan kapasitas paru.

3. Saran